<a href="https://colab.research.google.com/github/Charana04/Algorithmic-Trading-Strategy-Simulation/blob/main/Algorithmic_Trading_Strategy_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtrader


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.7 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import backtrader as bt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Fetch historical stock data
ticker = "AAPL"
data = yf.download(ticker, start="2010-01-01", end="2023-12-31")

# Simplify column headers
data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
print(data.head())  # Verify the column names

# Define Momentum Strategy
class MomentumStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=20)

    def next(self):
        if self.data.close[0] > self.sma[0]:
            self.buy()
        elif self.data.close[0] < self.sma[0]:
            self.sell()

# Define Mean-Reversion Strategy
class MeanReversionStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=20)
        self.upper_band = self.sma + (2 * bt.indicators.StandardDeviation(self.data.close, period=20))
        self.lower_band = self.sma - (2 * bt.indicators.StandardDeviation(self.data.close, period=20))

    def next(self):
        if self.data.close[0] < self.lower_band[0]:
            self.buy()
        elif self.data.close[0] > self.upper_band[0]:
            self.sell()

# Backtesting Function
def backtest_strategy(strategy, data, strategy_name):
    cerebro = bt.Cerebro()
    bt_feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(bt_feed)
    cerebro.addstrategy(strategy)

    print(f"\nRunning Backtest for: {strategy_name}")
    results = cerebro.run()

    # Calculate portfolio value at the end of the backtest
    portfolio_value = cerebro.broker.getvalue()

    # Calculate Daily Returns (after backtest is complete)
    daily_returns = pd.Series([cerebro.broker.getvalue() for _ in range(len(data))]).pct_change().dropna()

    # Calculate Annualized Return
    annualized_return = (portfolio_value / cerebro.broker.startingcash) ** (1 / (len(data) / 252)) - 1

    # Calculate Sharpe Ratio (assuming risk-free rate of 0%)
    mean_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()
    sharpe_ratio = mean_daily_return / std_daily_return * np.sqrt(252)

    print(f"Annualized Return: {annualized_return * 100:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

    # Plot results
    cerebro.plot()

    return cerebro

# Backtest Momentum Strategy
cerebro_momentum = backtest_strategy(MomentumStrategy, data, "Momentum Strategy")

# Backtest Mean-Reversion Strategy
cerebro_mean_reversion = backtest_strategy(MeanReversionStrategy, data, "Mean-Reversion Strategy")


[*********************100%***********************]  1 of 1 completed


            Adj Close     Close      High       Low      Open     Volume
Date                                                                    
2010-01-04   6.447412  7.643214  7.660714  7.585000  7.622500  493729600
2010-01-05   6.458560  7.656429  7.699643  7.616071  7.664286  601904800
2010-01-06   6.355827  7.534643  7.686786  7.526786  7.656429  552160000
2010-01-07   6.344076  7.520714  7.571429  7.466071  7.562500  477131200
2010-01-08   6.386254  7.570714  7.571429  7.466429  7.510714  447610800

Running Backtest for: Momentum Strategy


<ipython-input-8-5372dcc3403b>:61: RuntimeWarning: invalid value encountered in scalar divide
  sharpe_ratio = mean_daily_return / std_daily_return * np.sqrt(252)


Annualized Return: 15.66%
Sharpe Ratio: nan

Running Backtest for: Mean-Reversion Strategy
Annualized Return: -5.14+21.69j%
Sharpe Ratio: nan


In [ ]:
import pandas as pd

# Assuming 'df' is the DataFrame containing your data
print(data)


Ticker          AAPL       AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                                       
2010-01-04  6.447413   7.643214   7.660714   7.585000   7.622500  493729600
2010-01-05  6.458559   7.656429   7.699643   7.616071   7.664286  601904800
2010-01-06  6.355829   7.534643   7.686786   7.526786   7.656429  552160000
2010-01-07  6.344077   7.520714   7.571429   7.466071   7.562500  477131200
2010-01-08  6.386255   7.570714   7.571429   7.466429   7.510714  447610800
...              ...        ...        ...        ...        ...        ...
2010-12-23  9.748994  11.557143  11.612500  11.541786  11.607143  223157200
2010-12-27  9.781529  11.595714  11.622857  11.482857  11.530357  249816000
2010-12-28  9.805333  11.623929  11.666429  11.609286  11.639643  175924000
2010-12-29  9.799912  11.617500  11.658929  11.610714  11.650714  163139200
2010-12-30  9.750803  11.559286  11.625357  11.537500  11.624286  157494400

[251 rows x

In [ ]:
import yfinance as yf
import backtrader as bt
import pandas as pd
import numpy as np

# Fetch the data from Yahoo Finance
data = yf.download('AAPL', start='2010-01-01', end='2010-12-31')

# Drop the 'Ticker' level and reset the index if it's multi-level
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(0)

# Ensure that the 'Date' column is in datetime format if it's not already
data.index = pd.to_datetime(data.index)

# Define the strategy
class MomentumStrategy(bt.Strategy):
    def __init__(self):
        # Using a longer moving average period
        self.moving_avg = bt.indicators.SimpleMovingAverage(self.data.close, period=50)

    def next(self):
        # Log to check how the strategy is behaving
        print(f"Date: {self.datas[0].datetime.date(0)}, Close: {self.data.close[0]}, Moving Average: {self.moving_avg[0]}")

        # Trigger buy if the current close is above the moving average
        if self.data.close[0] > self.moving_avg[0]:
            if not self.position:  # Only buy if not already in a position
                self.buy()
        # Trigger sell if the current close is below the moving average
        elif self.data.close[0] < self.moving_avg[0]:
            if self.position:  # Only sell if already in a position
                self.sell()

# Add the PnL analyzer to track profit and loss
class MomentumWithPnLStrategy(MomentumStrategy):
    def __init__(self):
        super().__init__()

    def stop(self):
        # Print the final profit and loss (PnL) for the strategy
        print(f"Final PnL: {self.broker.get_value() - 100000}")

# Convert the data to Backtrader's PandasData feed
bt_data = bt.feeds.PandasData(dataname=data)

# Initialize cerebro
cerebro = bt.Cerebro()

# Add data and strategy to cerebro
cerebro.adddata(bt_data)
cerebro.addstrategy(MomentumWithPnLStrategy)

# Add a TradeAnalyzer
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# Set up the initial cash
cerebro.broker.set_cash(100000)

# Run the strategy
strat_results = cerebro.run()

# Extract the TradeAnalyzer results
trade_analyzer = strat_results[0].analyzers.trade_analyzer.get_analysis()

# Print the summary of trades
print(f"Total Number of Trades: {trade_analyzer.total['total']}")

if trade_analyzer.total['total'] > 0:
    print(f"Total Profit/Loss: {trade_analyzer.pnl.net}")

    # Extract the profit/loss for each trade
    pnl_results = [trade.pnl for trade in trade_analyzer.trades.values()]
    print(f"Profit/Loss from each trade: {pnl_results}")

    # Calculate the daily returns from the trade PnL results
    daily_returns = pd.Series(pnl_results).pct_change().dropna()

    # Calculate the Sharpe ratio
    mean_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()

    # Avoid NaN Sharpe ratio by checking for zero standard deviation
    if std_daily_return == 0:
        sharpe_ratio = 0
    else:
        sharpe_ratio = mean_daily_return / std_daily_return * np.sqrt(252)

    # Print the Sharpe ratio
    print(f"Sharpe Ratio: {sharpe_ratio}")
else:
    print("No trades were executed.")


[*********************100%***********************]  1 of 1 completed

Date: 2010-03-16, Close: nan, Moving Average: nan
Date: 2010-03-17, Close: nan, Moving Average: nan
Date: 2010-03-18, Close: nan, Moving Average: nan
Date: 2010-03-19, Close: nan, Moving Average: nan
Date: 2010-03-22, Close: nan, Moving Average: nan
Date: 2010-03-23, Close: nan, Moving Average: nan
Date: 2010-03-24, Close: nan, Moving Average: nan
Date: 2010-03-25, Close: nan, Moving Average: nan
Date: 2010-03-26, Close: nan, Moving Average: nan
Date: 2010-03-29, Close: nan, Moving Average: nan
Date: 2010-03-30, Close: nan, Moving Average: nan
Date: 2010-03-31, Close: nan, Moving Average: nan
Date: 2010-04-01, Close: nan, Moving Average: nan
Date: 2010-04-05, Close: nan, Moving Average: nan
Date: 2010-04-06, Close: nan, Moving Average: nan
Date: 2010-04-07, Close: nan, Moving Average: nan
Date: 2010-04-08, Close: nan, Moving Average: nan
Date: 2010-04-09, Close: nan, Moving Average: nan
Date: 2010-04-12, Close: nan, Moving Average: nan
Date: 2010-04-13, Close: nan, Moving Average: nan
